# 🚀 LSTM Algorithmic Trading Bot - Google Colab

**Полная система algorithmic trading с LSTM и Reinforcement Learning**

Этот notebook позволяет обучать модели на бесплатном GPU Google Colab.

## 📋 Что делает этот notebook:
- ✅ Автоматическая настройка GPU
- ✅ Клонирование и установка проекта
- ✅ Обучение LSTM модели
- ✅ Обучение RL агента
- ✅ Сравнение результатов
- ✅ Сохранение на Google Drive

## ⚡ Быстрый старт:
1. Нажмите `Runtime` → `Change runtime type` → `GPU`
2. Запустите все ячейки по порядку
3. Мониторьте обучение в логах

## 1️⃣ Проверка GPU и системной информации

In [ ]:
# Проверка доступности GPU
!nvidia-smi

# Системная информация
!cat /proc/cpuinfo | grep 'model name' | head -1
!free -h

# Версия Python
import sys
print(f"Python version: {sys.version}")

# Проверка CUDA
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Монтирование Google Drive (для сохранения моделей)

In [ ]:
# Монтирование Google Drive для сохранения результатов
from google.colab import drive
drive.mount('/content/drive')

# Создание папки для проекта
!mkdir -p /content/drive/MyDrive/LSTM_Trading_Models
print("✅ Google Drive смонтирован")
print("📁 Модели будут сохраняться в: /content/drive/MyDrive/LSTM_Trading_Models")

## 3️⃣ Клонирование репозитория и установка зависимостей

In [ ]:
# Клонирование репозитория
!git clone https://github.com/askimer/LSTM-Trading-Bot.git
%cd LSTM-Trading-Bot

# Установка зависимостей
!pip install -r requirements.txt

# Проверка установки
import torch
import stable_baselines3
import ccxt
print("✅ Все зависимости установлены")
print(f"PyTorch CUDA: {torch.cuda.is_available()}")
print(f"Stable Baselines3: {stable_baselines3.__version__}")

## 4️⃣ Создание демо-данных (если нет реальных)

In [ ]:
# Создание демо-данных для быстрого тестирования
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Генерация синтетических данных BTC
def create_demo_data():
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', periods=5000, freq='1min')
    
    # Базовая цена BTC с трендом и волатильностью
    base_price = 30000
    trend = np.linspace(0, 10000, 5000)  # Рост цены
    noise = np.random.normal(0, 500, 5000)  # Случайные колебания
    
    close_prices = base_price + trend + noise
    close_prices = np.maximum(close_prices, 1000)  # Минимум 1000
    
    # Генерация OHLCV
    high_prices = close_prices * (1 + np.abs(np.random.normal(0, 0.02, 5000)))
    low_prices = close_prices * (1 - np.abs(np.random.normal(0, 0.02, 5000)))
    open_prices = np.roll(close_prices, 1)
    open_prices[0] = close_prices[0]
    
    volume = np.random.uniform(10, 1000, 5000)
    
    df = pd.DataFrame({
        'timestamp': dates,
        'open': open_prices,
        'high': high_prices,
        'low': low_prices,
        'close': close_prices,
        'volume': volume
    })
    
    return df

# Создание демо-данных
demo_df = create_demo_data()
print(f"✅ Создано {len(demo_df)} строк демо-данных")
print(demo_df.head())

# Сохранение демо-данных
!mkdir -p btc_usdt_training_data
demo_df.to_csv('btc_usdt_training_data/full_btc_usdt_data_feature_engineered.csv', index=False)
print("💾 Демо-данные сохранены")

## 5️⃣ Быстрое обучение LSTM (демо-режим)

In [ ]:
# Модификация train_lstm.py для быстрого обучения в Colab
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from skorch import NeuralNetRegressor
from sklearn.model_selection import RandomizedSearchCV
from skorch.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
import time

# Определение модели LSTM
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(LSTMRegressor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

# Функция быстрого обучения
def quick_train_lstm():
    print("🚀 Начинаем быстрое обучение LSTM...")
    start_time = time.time()
    
    # Загрузка данных
    df = pd.read_csv('btc_usdt_training_data/full_btc_usdt_data_feature_engineered.csv')
    df = df.dropna()
    
    # Удаление константных столбцов
    std_dev = df.std()
    non_constant_columns = std_dev[std_dev != 0].index.tolist()
    df = df[non_constant_columns]
    
    X = df.drop('close', axis=1).values
    y = df['close'].values.reshape(-1, 1)
    
    # Масштабирование
    scaler_X = StandardScaler()
    X = scaler_X.fit_transform(X)
    
    scaler_y = StandardScaler()
    y = scaler_y.fit_transform(y)
    
    # Сохранение скейлеров
    with open('scaler_X.pkl', 'wb') as f:
        pickle.dump(scaler_X, f)
    with open('scaler_y.pkl', 'wb') as f:
        pickle.dump(scaler_y, f)
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Преобразование в тензоры
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"📊 Обучение на устройстве: {device}")
    
    n_features = X_train.shape[1]
    X_train_tensor = torch.tensor(np.array(X_train, copy=True, order='C').reshape(-1, 1, n_features), dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(np.array(y_train, copy=True, order='C'), dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(np.array(X_test, copy=True, order='C').reshape(-1, 1, n_features), dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(np.array(y_test, copy=True, order='C'), dtype=torch.float32).to(device)
    
    # Создание модели
    model = LSTMRegressor(input_dim=n_features, hidden_dim=128, num_layers=2)
    model = model.to(device)
    
    # Оптимизатор и loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # Обучение
    epochs = 50  # Быстрое обучение для демо
    batch_size = 64
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    print(f"🎯 Начало обучения: {epochs} эпох, batch_size={batch_size}")
    
    losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        epoch_loss /= len(train_loader)
        losses.append(epoch_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"📈 Эпоха {epoch+1}/{epochs}, Loss: {epoch_loss:.6f}")
    
    # Оценка модели
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        mse = criterion(predictions, y_test_tensor).item()
    
    print(f"✅ Обучение завершено! MSE: {mse:.6f}")
    print(f"⏱️ Время обучения: {time.time() - start_time:.1f} секунд")
    
    # Сохранение модели
    torch.save(model, 'lstm_model_colab.pt')
    torch.save(model.state_dict(), 'lstm_model_colab_state.pt')
    
    # Копирование на Google Drive
    !cp lstm_model_colab.pt /content/drive/MyDrive/LSTM_Trading_Models/
    !cp lstm_model_colab_state.pt /content/drive/MyDrive/LSTM_Trading_Models/
    !cp scaler_X.pkl /content/drive/MyDrive/LSTM_Trading_Models/
    !cp scaler_y.pkl /content/drive/MyDrive/LSTM_Trading_Models/
    
    print("💾 Модель сохранена на Google Drive")
    
    # График обучения
    plt.figure(figsize=(10, 6))
    plt.plot(losses)
    plt.title('LSTM Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.grid(True)
    plt.savefig('training_loss.png')
    plt.show()
    
    return model, mse

# Запуск обучения
model, final_mse = quick_train_lstm()

## 6️⃣ Обучение RL агента

In [ ]:
# Импорт необходимых модулей для RL
import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd
import torch
import time

# Определение Trading Environment
class TradingEnvironment(gym.Env):
    def __init__(self, df):
        super(TradingEnvironment, self).__init__()
        
        self.df = df.reset_index(drop=True)
        self.current_step = 0
        self.balance = 1000
        self.position = 0
        self.initial_balance = 1000
        
        # Actions: 0=Hold, 1=Buy, 2=Sell
        self.action_space = spaces.Discrete(3)
        
        # State: balance, position, price, indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0
        return self._get_state()
    
    def _get_state(self):
        if self.current_step >= len(self.df):
            return np.zeros(self.observation_space.shape)
        
        row = self.df.iloc[self.current_step]
        price = row['close']
        
        # Простые индикаторы
        sma5 = self.df.iloc[max(0, self.current_step-5):self.current_step+1]['close'].mean()
        rsi = 50  # Заглушка
        
        state = np.array([
            self.balance / self.initial_balance - 1,  # Нормализованный баланс
            self.position / (self.balance * 0.1) if self.balance > 0 else 0,  # Нормализованная позиция
            price / 50000 - 1,  # Нормализованная цена
            (price - sma5) / sma5 if sma5 > 0 else 0,  # Momentum
            rsi / 100 - 0.5  # RSI
        ], dtype=np.float32)
        
        return state
    
    def step(self, action):
        if self.current_step >= len(self.df) - 1:
            done = True
            reward = 0
            return self._get_state(), reward, done, {}
        
        current_price = self.df.iloc[self.current_step]['close']
        next_price = self.df.iloc[self.current_step + 1]['close']
        
        reward = 0
        done = False
        
        # Execute action
        if action == 1 and self.balance > current_price:  # Buy
            invest = min(self.balance * 0.1, self.balance)
            fee = invest * 0.00018
            btc_amount = (invest - fee) / current_price
            self.position += btc_amount
            self.balance -= invest
            reward -= 0.01  # Small penalty for trading
            
        elif action == 2 and self.position > 0:  # Sell
            sell_amount = self.position * 0.5
            revenue = sell_amount * current_price
            fee = revenue * 0.00018
            self.position -= sell_amount
            self.balance += revenue - fee
            reward -= 0.01  # Small penalty for trading
        
        # Portfolio change reward
        current_portfolio = self.balance + self.position * current_price
        next_portfolio = self.balance + self.position * next_price
        portfolio_change = (next_portfolio - current_portfolio) / current_portfolio if current_portfolio > 0 else 0
        reward += portfolio_change * 100
        
        # Penalty for large losses
        if current_portfolio < self.initial_balance * 0.8:
            reward -= 5
        
        self.current_step += 1
        if self.current_step >= len(self.df) - 1:
            done = True
        
        return self._get_state(), reward, done, {}

# Функция обучения RL
def train_rl_agent():
    print("🤖 Обучение RL агента...")
    
    # Загрузка данных
    df = pd.read_csv('btc_usdt_training_data/full_btc_usdt_data_feature_engineered.csv')
    df = df.dropna()
    
    # Создание среды
    env = DummyVecEnv([lambda: TradingEnvironment(df)])
    
    # Создание модели PPO
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        n_steps=1024,  # Уменьшено для Colab
        batch_size=32,
        n_epochs=5,
        gamma=0.99,
        ent_coef=0.01,
        device=device,
        verbose=1,
        tensorboard_log="./rl_tensorboard/"
    )
    
    # Обучение
    total_timesteps = 50000  # Уменьшено для быстрого обучения
    print(f"🎯 Обучение RL: {total_timesteps} шагов")
    
    start_time = time.time()
    model.learn(total_timesteps=total_timesteps, progress_bar=True)
    training_time = time.time() - start_time
    
    print(f"✅ RL обучение завершено за {training_time:.1f} секунд")
    
    # Сохранение модели
    model.save("ppo_trading_agent_colab")
    
    # Копирование на Google Drive
    !cp ppo_trading_agent_colab.zip /content/drive/MyDrive/LSTM_Trading_Models/ 2>/dev/null || true
    
    print("💾 RL модель сохранена")
    
    return model

# Обучение RL агента
rl_model = train_rl_agent()

## 7️⃣ Сравнение результатов LSTM vs RL

In [ ]:
# Сравнение производительности моделей
import matplotlib.pyplot as plt

def evaluate_models():
    print("📊 Сравнение моделей...")
    
    # Загрузка тестовых данных
    df = pd.read_csv('btc_usdt_training_data/full_btc_usdt_data_feature_engineered.csv')
    df = df.dropna()
    test_df = df.tail(500)  # Последние 500 точек для тестирования
    
    # Оценка LSTM
    lstm_model = torch.load('lstm_model_colab.pt', map_location='cpu')
    lstm_model.eval()
    
    # Подготовка данных для LSTM
    with open('scaler_X.pkl', 'rb') as f:
        scaler_X = pickle.load(f)
    
    # Простая оценка (демо)
    lstm_predictions = []
    actual_prices = []
    
    for i in range(len(test_df)):
        row = test_df.iloc[i]
        # Упрощенная подготовка признаков
        features = np.array([row.get('close', 30000), row.get('volume', 100), 0, 0, 0, 0, 0, 0])
        features_scaled = scaler_X.transform(features.reshape(1, -1))
        
        input_tensor = torch.tensor(features_scaled.reshape(1, 1, -1), dtype=torch.float32)
        with torch.no_grad():
            pred = lstm_model(input_tensor).item()
        
        lstm_predictions.append(pred)
        actual_prices.append(row['close'])
    
    # Оценка RL
    rl_env = TradingEnvironment(test_df)
    rl_model = PPO.load("ppo_trading_agent_colab")
    
    rl_rewards = []
    state = rl_env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action, _ = rl_model.predict(state, deterministic=True)
        state, reward, done, _ = rl_env.step(action)
        total_reward += reward
        rl_rewards.append(reward)
    
    # Вывод результатов
    print("\n" + "="*50)
    print("📈 СРАВНЕНИЕ МОДЕЛЕЙ")
    print("="*50)
    print(f"LSTM - MSE на тесте: {np.mean((np.array(lstm_predictions) - np.array(actual_prices))**2):.2f}")
    print(f"RL - Общая награда: {total_reward:.2f}")
    print(f"RL - Финальный портфель: ${rl_env.balance + rl_env.position * test_df.iloc[-1]['close']:.2f}")
    
    # График сравнения
    plt.figure(figsize=(15, 10))
    
    # LSTM предсказания vs реальные цены
    plt.subplot(2, 2, 1)
    plt.plot(actual_prices[-100:], label='Реальные цены', alpha=0.7)
    plt.plot(lstm_predictions[-100:], label='LSTM предсказания', alpha=0.7)
    plt.title('LSTM: Предсказания vs Реальность')
    plt.legend()
    plt.grid(True)
    
    # RL награды
    plt.subplot(2, 2, 2)
    plt.plot(rl_rewards)
    plt.title('RL: Награды по шагам')
    plt.grid(True)
    
    # Распределение наград RL
    plt.subplot(2, 2, 3)
    plt.hist(rl_rewards, bins=50, alpha=0.7)
    plt.title('RL: Распределение наград')
    plt.grid(True)
    
    # Сводка
    plt.subplot(2, 2, 4)
    models = ['LSTM', 'RL']
    scores = [
        1 / (1 + np.mean((np.array(lstm_predictions) - np.array(actual_prices))**2)),  # Инвертированная MSE
        min(1.0, total_reward / 1000)  # Нормализованная награда
    ]
    plt.bar(models, scores, alpha=0.7)
    plt.title('Сравнение производительности')
    plt.ylabel('Относительная оценка')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Копирование результатов на Drive
    !cp model_comparison.png /content/drive/MyDrive/LSTM_Trading_Models/ 2>/dev/null || true
    
    print("\n✅ Сравнение завершено!")
    print("📊 Результаты сохранены в model_comparison.png")

# Запуск сравнения
evaluate_models()

## 8️⃣ Финализация и сохранение результатов

In [ ]:
# Финализация сессии
import datetime

print("\n" + "="*60)
print("🎉 GOOGLE COLAB SESSION COMPLETED!")
print("="*60)

# Информация о сохраненных файлах
print("\n📁 СОХРАНЕННЫЕ ФАЙЛЫ:")
print("Google Drive: /MyDrive/LSTM_Trading_Models/")
print("- lstm_model_colab.pt (LSTM модель)")
print("- ppo_trading_agent_colab.zip (RL модель)")
print("- scaler_X.pkl, scaler_y.pkl (скейлеры)")
print("- model_comparison.png (сравнение моделей)")
print("- training_loss.png (график обучения LSTM)")

# Статистика сессии
end_time = datetime.datetime.now()
print(f"\n⏰ Время завершения: {end_time}")
print(f"🎯 GPU использован: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"💻 GPU: {torch.cuda.get_device_name(0)}")
    print(f"🧠 GPU память: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Рекомендации
print("\n💡 РЕКОМЕНДАЦИИ:")
print("1. Скачайте модели с Google Drive")
print("2. Используйте их локально с live_trading.py")
print("3. Для продолжения обучения - запустите notebook снова")
print("4. Мониторьте лимиты Colab (12 часов, 90 мин бездействия)")

print("\n🚀 УДАЧИ В АЛГОРИТМИЧЕСКОЙ ТОРГОВЛЕ!")
print("="*60)